##### Import libraries

In [5]:
import ast
import string
import random
from faker import Faker
import pandas as pd

##### Load historical data from .csv file

In [6]:
resp_df = pd.read_csv('baccarat_records.csv').drop(columns=['Unnamed: 0'])

# Parse player_hand and banker_hand
resp_df['player_hand'] = resp_df['player_hand'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
resp_df['banker_hand'] = resp_df['banker_hand'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
resp_df.head()

# Create exploded data
player_hand_df = resp_df[['game_id', 'player_id', 'player_hand']].explode('player_hand').reset_index(drop=True)
banker_hand_df = resp_df[['game_id', 'player_id', 'banker_hand']].explode('banker_hand').reset_index(drop=True)

##### Build `game` table

In [ ]:
game_df = resp_df[['game_id', 'game_name', 'status', 'start_time', 'end_time', 'last_action',
                   'player_wager', 'player_payout', 'game_outcome', 'player_bet', 'player_bet_outcome', 'player_id'
                   ]]

game_df.head()

##### Build `player_hand` table

In [7]:
# Normalize exploded data
player_hand_norm_df = pd.json_normalize(player_hand_df['player_hand'])
player_hand_norm_df.head()

# Concatenate with player_hand
player_hand_df_final = pd.concat([player_hand_df, player_hand_norm_df], axis = 1).drop(columns = ['player_hand'])
player_hand_df_final.head()

# Add hand count
card_count = player_hand_df_final.groupby('game_id').cumcount() + 1
player_hand_df_final.insert(1, 'player_hand_count', card_count) 
player_hand_df_final['player_hand_count'] = player_hand_df_final['player_hand_count'].apply(lambda x: str(x).rjust(2, '0'))
player_hand_df_final.head(10)

# Add player_hand_id
ph_id = player_hand_df_final['game_id'] + '-' + player_hand_df_final['player_hand_count']
player_hand_df_final.insert(0, 'player_hand_id', ph_id) 

# Display
player_hand_df_final.head()

,player_hand_id,game_id,player_hand_count,player_id,value,rank
0,GID-ZHLCRX-01,GID-ZHLCRX,01,PID-000028,5,Diamonds
1,GID-ZHLCRX-02,GID-ZHLCRX,02,PID-000028,6,Clubs
2,GID-ZHLCRX-03,GID-ZHLCRX,03,PID-000028,10,Diamonds
3,GID-U3GQXH-01,GID-U3GQXH,01,PID-000041,5,Clubs
4,GID-U3GQXH-02,GID-U3GQXH,02,PID-000041,8,Spades


##### Build `banker_hand` table

In [8]:
# Normalize exploded data
banker_hand_norm_df = pd.json_normalize(banker_hand_df['banker_hand'])
banker_hand_norm_df.head()

# Concatenate with banker_hand
banker_hand_df_final = pd.concat([banker_hand_df, banker_hand_norm_df], axis = 1).drop(columns = ['banker_hand'])
banker_hand_df_final.head()

# Add hand count
card_count = banker_hand_df_final.groupby('game_id').cumcount() + 1
banker_hand_df_final.insert(1, 'banker_hand_count', card_count) 
banker_hand_df_final['banker_hand_count'] = banker_hand_df_final['banker_hand_count'].apply(lambda x: str(x).rjust(2, '0'))
banker_hand_df_final.head(10)

# Add banker_hand_id
bh_id = banker_hand_df_final['game_id'] + '-' + banker_hand_df_final['banker_hand_count']
banker_hand_df_final.insert(0, 'banker_hand_id', bh_id) 

# Display
banker_hand_df_final.head()

,banker_hand_id,game_id,banker_hand_count,player_id,value,rank
0,GID-ZHLCRX-01,GID-ZHLCRX,01,PID-000028,6,Diamonds
1,GID-ZHLCRX-02,GID-ZHLCRX,02,PID-000028,5,Clubs
2,GID-ZHLCRX-03,GID-ZHLCRX,03,PID-000028,2,Spades
3,GID-U3GQXH-01,GID-U3GQXH,01,PID-000041,Q,Diamonds
4,GID-U3GQXH-02,GID-U3GQXH,02,PID-000041,6,Spades


##### Build `hand_value` table

In [9]:
hand_value_df = resp_df[['game_id', 'player_hand_value_df', 'banker_hand_value_df']]

hvid = ['HVID' + '-' +''.join(random.choices(string.ascii_uppercase + string.digits, k=6)) for _ in range(100000)]
hand_value_df.insert(0, 'hand_value_id_df', hvid)

hand_value_df.head()

,hand_value_id,game_id,player_hand_value,banker_hand_value
0,HVID-H5X9DL,GID-ZHLCRX,1,3
1,HVID-IBVSWJ,GID-U3GQXH,5,6
2,HVID-HLCJSK,GID-QNA9SC,8,8
3,HVID-5F02QK,GID-QBI1SH,0,8
4,HVID-K3FVQE,GID-UTZLT1,6,8


In [10]:
transaction_df = resp_df[['game_id', 'player_id', 'player_wager', 'player_payout', 'end_time']]
transaction_df.head()

tid = ['TID' + '-' +''.join(random.choices(string.ascii_uppercase + string.digits, k=6)) for _ in range(100000)]
transaction_df.insert(0, 'transaction_id_df', tid)
transaction_df = transaction_df.rename(columns = {'end_time':'transaction_time_df'})

transaction_df.head()

,transaction_id,game_id,player_id,player_wager,player_payout,transaction_time
0,TID-PCIAUB,GID-ZHLCRX,PID-000028,108,-108.0,2023-01-16T10:20:12
1,TID-2HG31B,GID-U3GQXH,PID-000041,361,-361.0,2023-09-05T07:20:55
2,TID-7C7Q1Y,GID-QNA9SC,PID-000041,707,-707.0,2023-09-01T00:45:52
3,TID-VENP6N,GID-QBI1SH,PID-000017,638,-638.0,2023-03-19T22:46:39
4,TID-GVSVPZ,GID-UTZLT1,PID-000047,15,-15.0,2023-02-26T02:43:11


##### Build `player` table

In [14]:
fake = Faker()

player_id = [f'PID-{i:06}' for i in range(1, 51)]
player_name = [fake.name() for _ in range(len(player_id))]

beginning_balance = [random.choice([25000, 50000, 100000, 200000]) for _ in range(len(player_id))]
total_payouts = transaction_df.groupby('player_id')['player_payout'].sum().reset_index(drop = True)

player_df = pd.DataFrame(
    {'player_id': player_id,
     'player_name': player_name,
     'beginning_balance': beginning_balance,
     'total_payouts': total_payouts
     }
)

player_df['current_balance'] = player_df['beginning_balance'] + player_df['total_payouts']

player_df.head()

,player_id,player_name,beginning_balance,total_payouts,current_balance
0,PID-000001,Chad Burgess,50000,-104279.50,-54279.50
1,PID-000002,James Santana,200000,-90326.50,109673.50
2,PID-000003,Makayla Palmer,25000,9246.75,34246.75
3,PID-000004,Jody Morris,200000,-41774.65,158225.35
4,PID-000005,Mr. Marcus Reed,25000,-115953.05,-90953.05


In [15]:
# player_df.to_csv('toSQL_player_table.csv')
# game_df.to_csv('toSQL_game_table.csv')
# player_hand_df_final.to_csv('toSQL_player_hand_table.csv')
# banker_hand_df_final.to_csv('toSQL_banker_hand_table.csv')
# hand_value_df.to_csv('toSQL_hand_value_table.csv')
# transaction_df.to_csv('toSQL_transaction_table.csv')